In [1]:
import nltk
from nltk.corpus import *
from nltk.stem.porter import *
import pickle
from pathlib import Path


In [2]:
# stemming porter object
stemmer = PorterStemmer()
stop_words = set(stopwords.words('english'))
root = Path("../")

In [3]:
my_path = root / "Pickled_files" / "Inverted_index"
dbfile = open(my_path, 'rb')     
inverted_idx = pickle.load(dbfile)
dbfile.close()

my_path = root / "Pickled_files" / "Documents"
dbfile = open(my_path, 'rb')     
documents = pickle.load(dbfile)
dbfile.close()

my_path = root / "Pickled_files" / "Files"
dbfile = open(my_path, 'rb')     
files_list = pickle.load(dbfile)
dbfile.close()

In [4]:
def boolean_retrieval(query , type) : 
    ans = list()
    tokens = nltk.tokenize.word_tokenize(str(query))
    new_token = list()
    for i in tokens:
        cur_str = i.lower()
        if cur_str in stop_words or cur_str.isnumeric(): 
            continue
        new_token.append(cur_str)
    tokens = new_token
    rank , file_id = dict() , dict()
    set_id = list()
    for words in tokens : 
        freq = dict()
        normalized_word = stemmer.stem(words)
        if inverted_idx.get(normalized_word) == None : 
            return ans
        ids = set()
        for files in inverted_idx[normalized_word] : 
            ids.add(files[0])
            file_id[files[0]] = files[1]
            if rank.get(files[0]) == None : 
                rank[files[0]] = files[2]
            else : 
                rank[files[0]] += files[2]
        set_id.append(ids)

    if type :   
        merged_id = list(set.intersection(*set_id))
    else :
        merged_id = list(set.union(*set_id))
    merged_id = sorted(merged_id , key = lambda y : -rank[y])
    for i in range(0 , min(3 , len(merged_id))) : 
        ans.append((documents[merged_id[i]] , files_list[file_id[merged_id[i]]]))
    return ans

In [5]:
and_query = boolean_retrieval("Illegal Drugs ComPensation" , True)
for i in range(len(and_query)) : 
    print(str(i + 1) + ") " , and_query[i][1])
    print(and_query[i][0])
    print("\n\n\n---------------------\n\n\n")
    

1)  .\Docs\Property\BRIT-PO-Policy-Wording-May-2016-1.docx
('any punitive or exemplary damages , compensations , fines or any penalties of whatsoever nature which the insured is ordered to pay by a forum , authority or body of competent jurisdiction ; in respect of coverage clause 1 eviction of squatters only : - a legal expenses incurred in relation to any dispute where the cause of action involves the insured ’ s legitimate tenant ; b any claim resulting from the occupation of the insured premises or part thereof by squatters prior to the inception of this policy ; c any action consciously taken by the insured that hinders the insurer or appointed representative or adversely affects the course of the legal proceedings initiated for the eviction of squatters ; in respect of coverage clause 5 criminal proceedings only , arising out of any criminal proceedings or allegations in respect of : the ownership , possession of or use of any vehicle ; or any investigation by hmrc or the departm